In [2]:
import pandas as pd
import numpy as np

from datetime import datetime

import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
from plotly.subplots import make_subplots


from sqlalchemy import create_engine


In [3]:
HOST = '37.139.42.145'
DBNAME = 'game-analytics'
USER = 'analytics'
PASSWORD = 'BRtTaqYiJyr29WXN'
TABLE_SCHEMA = 'diploma'
ENGINE = create_engine(f'postgresql://{USER}:{PASSWORD}@{HOST}/{DBNAME}')


# ROI маркетинг по формуле (ROI=revenue(iap + ad) / spend)

In [9]:
# ROI маркетинг по формуле (ROI=revenue(iap + ad) / spend) за период  с '2022-04-01' по '2022-05-01'таблица .geo_daily_report

start_time = datetime.now()

data_start ='2022-04-01'
data_end ='2022-05-01'
data_project_id = 'MiT'

roi_sql_geo_daily_report = pd.read_sql(f'''
SELECT

    (SUM(af_ad_revenue_usd) + SUM(af_purchase_usd)) /SUM(total_cost) AS roi
FROM {TABLE_SCHEMA}.geo_daily_report 
WHERE
    1=1
    AND project_id = '{data_project_id}' 
    AND event_date >='{data_start}'
    AND event_date < '{data_end}'
ORDER BY 1
''', con=ENGINE)

print('execution time: %s' % str(datetime.now() - start_time))
roi_sql_geo_daily_report.head(2)

execution time: 0:00:00.193073


,roi
0,1.305818


# 3.1.Какие именно рекламные кампании, из каких сеток зарекомендовали себя с
# лучшей стороны? Обосновать критерий качества трафика.

In [19]:
# ROI по media_source  по формуле (ROI=revenue(ad) / spend) за период  с '2022-04-01' по '2022-05-01'таблица .geo_daily_report

start_time = datetime.now()

data_start ='2022-04-01'
data_end ='2022-05-01'
data_project_id = 'MiT'

roi_media_source_sql_geo = pd.read_sql(f'''
SELECT
    media_source,    
    ROUND((SUM(af_ad_revenue_usd) /SUM(total_cost))::numeric,4) AS roi
FROM {TABLE_SCHEMA}.geo_daily_report 
WHERE
    1=1
    AND project_id = '{data_project_id}' 
    AND event_date >='{data_start}'
    AND event_date < '{data_end}'
GROUP BY 1
ORDER BY roi DESC NULLS LAST
''', con=ENGINE)

print('execution time: %s' % str(datetime.now() - start_time))
roi_media_source_sql_geo

execution time: 0:00:00.192769


,media_source,roi
0,bytedanceglobal_int,0.0809
1,Facebook Ads,0.0340
2,Organic,NaN
3,googleadwords_int,NaN


In [4]:
# ROI маркетинг по campaign (ROI=revenue(ad) / spend) за период  с '2022-04-01' по '2022-05-01'таблица .geo_daily_report

start_time = datetime.now()

data_start ='2022-04-01'
data_end ='2022-05-01'
data_project_id = 'MiT'

roi_campaign_sql_geo = pd.read_sql(f'''
SELECT
    campaign,    
    ROUND((SUM(af_ad_revenue_usd) /SUM(total_cost))::numeric,4) AS roi
FROM {TABLE_SCHEMA}.geo_daily_report 
WHERE
    1=1
    AND project_id = '{data_project_id}' 
    AND event_date >='{data_start}'
    AND event_date < '{data_end}'
GROUP BY 1
ORDER BY roi DESC NULLS LAST
''', con=ENGINE)

print('execution time: %s' % str(datetime.now() - start_time))
roi_campaign_sql_geo.head()

execution time: 0:00:01.061443


,campaign,roi
0,MiT_EM_AND_SEA_MAI_IT1_2803,0.0924
1,MiT_EM_AND_US_Purchase_IT1_2703,0.0906
2,MiT_EM_AND_SEA_Purchase_IT2_0604,0.0587
3,MiT_MS_AND_US_PURCH_AEO_IT1_1504,0.0528
4,MiT_EM_AND_T1_Purchase_IT2_0604,0.0496


# 3.2.Какие именно рекламные кампании, из каких сеток эффективны с точки зрения
# рекламной монетизации?

In [ ]:
лучшее соотношение ads и cpi

In [6]:
#cpi&ads по media_sorce geo_daily_report

cohort_start ='2022-04-01'
cohort_end ='2022-05-01'
project_id = 'MiT'

cpi_rev_geo_daily_report_cpi_sql = pd.read_sql(f'''       
SELECT  
    media_source,
    ROUND((SUM(af_ad_revenue_usd) / (SUM(total_cost) / sum(installs)))::numeric,4) as ads_cpi
FROM 
    {TABLE_SCHEMA}.geo_daily_report     
WHERE 
    1 = 1
    AND project_id = '{project_id}' 
    AND event_date >='{cohort_start}'
    AND event_date < '{cohort_end}'
GROUP BY 1
ORDER BY 2 DESC NULLS LAST
''', con=ENGINE)
cpi_rev_geo_daily_report_cpi_sql

,media_source,ads_cpi
0,bytedanceglobal_int,535.5559
1,Facebook Ads,4.9591
2,Organic,NaN
3,googleadwords_int,NaN


In [ ]:
С учетом имющихся данных лучший по каналу это Facebook Ads

In [7]:
#ads и cpi по media_sorce geo_daily_report

cohort_start ='2022-04-01'
cohort_end ='2022-05-01'
project_id = 'MiT'

cpi_rev_geo_daily_report_cpi_sql = pd.read_sql(f'''
SELECT  
    campaign,
    ROUND((SUM(af_ad_revenue_usd) / (SUM(total_cost) / sum(installs)))::numeric,4) as ads_cpi
FROM 
    {TABLE_SCHEMA}.geo_daily_report        
WHERE 
    1 = 1
    AND project_id = '{project_id}' 
    AND event_date >='{cohort_start}'
    AND event_date < '{cohort_end}'
GROUP BY 1
ORDER BY 2 DESC NULLS LAST
''', con=ENGINE)
cpi_rev_geo_daily_report_cpi_sql.head()

,campaign,ads_cpi
0,MiT_EM_AND_US_Purchase_IT1_2703,286.7010
1,MiT_EM_AND_SEA_MAI_IT1_2803,179.8743
2,MiT_EM_AND_T1_Purchase_IT2_0604,46.9770
3,MiT_EM_AND_SEA_Purchase_IT2_0604,32.3536
4,MiT_MS_AND_T1_PURCH_AEO_IT1_2203,1.5590


# 3.3.Какие именно рекламные кампании, из каких сеток эффективны с точки зрения
# покупок внутри приложений?

In [ ]:
лучшее соотношение cpi и iap

In [10]:
#cpi&iap по media_sorce geo_daily_report

cohort_start ='2022-04-01'
cohort_end ='2022-05-01'
project_id = 'MiT'

cpi_rev_geo_daily_report_cpi_sql = pd.read_sql(f'''       
SELECT  
    media_source,
    ROUND((SUM(af_purchase_usd) / (SUM(total_cost) / sum(installs)))::numeric,4) as ads_cpi
FROM 
    {TABLE_SCHEMA}.geo_daily_report     
WHERE 
    1 = 1
    AND project_id = '{project_id}' 
    AND event_date >='{cohort_start}'
    AND event_date < '{cohort_end}'
GROUP BY 1
ORDER BY 2 DESC NULLS LAST
''', con=ENGINE)
cpi_rev_geo_daily_report_cpi_sql

,media_source,ads_cpi
0,bytedanceglobal_int,2672.3376
1,Facebook Ads,25.6295
2,Organic,NaN
3,googleadwords_int,NaN


In [11]:
#cpi&iap по campaign geo_daily_report

cohort_start ='2022-04-01'
cohort_end ='2022-05-01'
project_id = 'MiT'

cpi_rev_geo_daily_report_cpi_sql = pd.read_sql(f'''
SELECT  
    campaign,
    ROUND((SUM(af_purchase_usd) / (SUM(total_cost) / sum(installs)))::numeric,4) as ads_cpi
FROM 
    {TABLE_SCHEMA}.geo_daily_report        
WHERE 
    1 = 1
    AND project_id = '{project_id}' 
    AND event_date >='{cohort_start}'
    AND event_date < '{cohort_end}'
GROUP BY 1
ORDER BY 2 DESC NULLS LAST
''', con=ENGINE)
cpi_rev_geo_daily_report_cpi_sql.head()

,campaign,ads_cpi
0,MiT_EM_AND_US_Purchase_IT1_2703,1328.0898
1,MiT_EM_AND_SEA_MAI_IT1_2803,1051.0255
2,MiT_EM_AND_T1_Purchase_IT2_0604,341.7651
3,MiT_EM_AND_SEA_Purchase_IT2_0604,162.4092
4,MiT_MS_AND_T1_PURCH_AEO_IT1_2203,11.3690
